# Analytical Sliding Block Solutions
Creates (mostly) analytical solutions for rigid sliding block analyses with harmonic input.

## Purpose
During the development of the rigid block analysis for pyslammer, we discovered that the integration scheme we used was different than the one originally used in SLAMMER.
The main difference is that our approach integrated the full acceleration, velocity, and displacement signals of the base and block, only finding the differential displacement at the end.
Let's call this the "absolute integration approach" (AIA).
SLAMMER's integration scheme subtracts the block acceleration from the base acceleration and integrates the $\delta$ acceleration to find the $\delta$ velocity and $\delta$ displacement.
Let's call this the "difference integration approach" (DIA).
The DIA requires less computation, but also doesn't produce some of the resulting arrays that we'd like to see in the analysis output (e.g., absolute block and velocity).
Even though DIA doesn't produce those results by default, it could be altered to produce them.

Another difference between the methods is the numerical integration approach. 
Our AIA uses Velocity Verlet integration.
SLAMMER's DIA uses trapezoidal integration.
These differences are not inherent in the DIA/AIA differences.

The larger issue is that DIA and AIA produce slightly different results that accumulate with increasing input acceleration to yield acceleration ratio, frequency, and duration.
We suspect that AIA is a better approach than DIA, but we need a "correct" answer to compare both models to in order to confirm or deny our guess.

The analytical solutions for rigid block analysis with harmonic input will provide the basis for judging whether DIA or AIA are more accurate.

### Why Bother?
Sliding block methods are known to be poor representations of the physical displacements of slopes during earthquakes, so they're really only used as an idex of displacement.
One could argue that, since SLAMMER has been in use for many years and indicies are used to predict *relative* performance, it's more important for pyslammer to replicate SLAMMER's results than an analytical solution.

So why even bother comparing these integration methods?
The first reason is that it's interesting.
The second reason, which actually addresses the reasoning behind the question, is that sliding block methods, flawed as they are, are still widely used in the earthquake geotechnical engineering community and continue to be developed and improved. 
We hope that improvements to sliding block methods from the engineering community will be incorporated into pyslammer through open-source contributions.
Any such contributions should be able to take advantage of the best computational approximations available.
However, even if pyslammer's default integration approach ends up varying from SLAMMER's, an optional "compatability mode" will be availalbe using DIA for use cases where comparison with previous SLAMMER output is important.


## Approach
The analytical model uses one cycle of a sine wave acceleration input ($\ddot{u}_{base}$).
The sine wave has input frequency, $f$ (Hz):

$\ddot{u}_{base}(t) = \sin 2 \pi f t $

The start of sliding occurs at $t_1$, when $\ddot{u}_{base}$ is equal to the yield acceleration, $k_y$. 
By definition, the slope of the input velocity signal ($\dot{u}_{base}$) at this time is $\ddot{u}_{base}(t_1) = k_y$.
With this slope, and the cordinate ($t_1$,$\dot{u}_{base}(t_1)$), we can find the equation for the block velocity ($\dot{u}_{block}$) as:

$\dot{u}_{block}(t) = g k_y (t - t_1) - \dot{u}_{base}(t_1)$

The end of sliding occurs at $t_2$, when $\dot{u}_{block}(t)$ is equal to $\dot{u}_{base}(t)$. 
Unfortunately, solving for $t_2$ is non-trivial because it involves a transendental equation (it essentially boils down to $x = \cos x$).
The intersection between $\dot{u}_{block}(t)$ and $\dot{u}_{base}(t)$ can only be approximated numerically, which means the "analytical" solution presented here isn't truly analytical.
But since we'll be solving with a numerical solver with a tolerance of $1e^{-15}$ seconds, and the AIA and DIA models typically work with signals on the order of $1e^{-3}$ seconds, the numerical portion of the "analytical" sliding block doesn't disqualify it from acting as a basis of comparing the AIA and DIA models.

The analytical model only provides the accumulated displacement for a single cycle of motion.
When compared to DIA and AIA simulations with longer duration, the results should be multiplied by the number of cycles in the longer simulation.
The comparison is only valid if the number of cycles in the longer simulation is an integer value.

### Parameters
Two parameters control the analytical model behavior: $f$ and $k_y$.

For comparison with numeric integration results, the analytical model solution is computed for:

$f$ = `[0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`

and

$k_y$ = `[0, 0.2, 0.4, 0.6, 0.8, 1.0]`



## Results
Unknown...


In [17]:
%matplotlib widget

In [18]:
# Import libraries
import pickle as pkl
import numpy as np
import sympy as sym
import pandas as pd
import matplotlib.pyplot as plt

sym.init_printing()

In [19]:
# Set up the parametric space

# for analytical model
frequencies = np.array([0.5,1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
yield_accelerations = np.array([0.1,0.2,0.4,0.6,0.8,1.0])
combinations = np.array(np.meshgrid(frequencies, yield_accelerations)).T.reshape(-1, 2)

# for numerical models
sample_resolution = np.array([50, 100, 200, 250, 500, 750, 1000]) # samples per second

In [20]:
# Define symbols for analytical solution
# Three kinds of symbols are used:
# 1. Function and parameter names: the names of functions and parameters. These are used purely for display purposes.
#    Function names have a single underscore at the beginning of the name (e.g. _abase)
# 2. Function expressions: the definitions of functions using parameters. These are used to evaluate the functions.
# 3. Parameters: variables and constants used to define functions


# # Define the function names
# _a, _v, _v_max, _vblock = sym.symbols('\ddot{u}_{in}(t)/g, \dot{u}_{in}(t), ||\dot{u}_{in\,max}||, \dot{v}_{block}(t)')
# # Define parameter names
# _t1, _t2, _g, _ky, _f = sym.symbols('t_1, t_2, g, k_y, f')
# # display(_a, _v, _v_max, _vblock)

# Define parameters
time, freq, g, ky = sym.symbols('t, f, g, k_y', real=True, positive=True)
# time, freq, g, ky, t1, t2 = sym.symbols('t, f, g, k_y, t_1, t_2', real=True, positive=True)
# display(time, freq, g, ky, t1, t2)

# Define the function expressions
a_in = sym.sin(freq*2*sym.pi*time)
v_inmax = g/2/sym.pi/freq
v_in = g*sym.integrate(a_in, time) + v_inmax
x_in = sym.integrate(v_in, time)

def show_solution_approach():
     display("The input acceleration (in units of g) is defined by:")
     display(sym.Eq(sym.symbols('\ddot{u}_{in}(t)/g'), a_in))
     display("The input velocity (in m/s) is defined by:")
     display(sym.Eq(sym.symbols('\dot{u}_{in}(t)'), v_in))
     display("The input displacement (in m) is defined by:")
     display(sym.Eq(sym.symbols('u_{in}(t)'), x_in))
     pass

def create_harmonic_input_files(freq, resolution, cycles=10):
    duration = cycles/freq
    t = np.linspace(0, duration, int(duration*resolution*cycles))
    a = np.sin(np.pi*2*freq*t)
    data = np.column_stack((t, a))
    np.savetxt(f'sample_ground_motions/sine_{freq}_Hz_{resolution*freq}_sps.csv', data, delimiter=',', header=f'# Time Series: {freq} Hz sine wave \n # Time (s),Acceleration (g)')


def find_harmonic_solution(freq_val, ky_val, grav = 9.81, plot=True):
        vals = [(g,grav),(ky,ky_val), (freq,freq_val)]

        # a = sym.sin(freq*2*sym.pi*time)
        # vo = g/2/sym.pi/freq#sym.symbols('v_o')#g*2*sym.pi
        # v = g*sym.integrate(a, time) + vo
        t1 = find_t1(a_in, ky, vals)

        vb = g*ky*(time-t1) + v_in.subs(time, t1)

        t2 = find_t2(v_in, vb, vals, freq_val)

        dx_block = find_displacement(v_in, vb, t1, t2, vals)
        total_disp = float(dx_block.subs(time, t2).subs(vals))
        if plot:
            harmonic_solution_plot(a_in, v_in, vb, dx_block, time, t1, t2, ky, vals)

        
        return total_disp

def find_t1(a_in, ky, vals):
    t1 = sym.solve(a_in-ky, time)[1]
    return t1.subs(vals)

def find_t2(v_in, vb, vals, freq_val):
    equation = sym.Eq(vb, v_in).subs(vals)
    t2 = sym.nsolve(equation, time, (0.25/freq_val, 1/freq_val), solver='bisect')
    return t2

def find_displacement(v_in, vb, t1, t2, vals):
    x_in = sym.integrate(v_in, time)
    x_block = sym.integrate(vb, time) + x_in.subs(time,t1) - sym.integrate(vb, time).subs(time,t1)
    total_displacement = x_in.subs(vals).subs(time,t2) - x_block.subs(vals).subs(time,t2)
    displacement = sym.Piecewise((0, time < t1), (total_displacement, time > t2),(x_in - x_block, True))
    # m_to_cm = 0.01
    return displacement

def apply_find_harmonic_solution(row, plot = False):
    return find_harmonic_solution(row['Frequency (Hz)'], row['ky (g)'], plot=plot)

def harmonic_solutions(harmonic_combinations, save=False, plot=False):
    harmonic_solutions = pd.DataFrame(harmonic_combinations, columns=['Frequency (Hz)', 'ky (g)'])
    harmonic_solutions['Displacement (m)'] = harmonic_solutions.apply(lambda row: apply_find_harmonic_solution(row, plot=plot), axis=1)
    if save:
        harmonic_solutions.to_csv('common/harmonic_solutions.csv', index=False)
    return harmonic_solutions

def harmonic_solution_plot(a_in, v_in, vb, displacement,time,t1_def_val,t2_val, ky, vals, save=False):
    ablock = sym.Piecewise((a_in, time < t1_def_val), (a_in, time > t2_val),(ky, True))
    aplot = sym.lambdify(time, a_in.subs(vals), 'numpy')
    abplot = sym.lambdify(time, ablock.subs(vals), 'numpy')
    vplot = sym.lambdify(time, v_in.subs(vals), 'numpy')
    vbplot = sym.lambdify(time, vb.subs(vals), 'numpy')
    
    displacementplot = sym.lambdify(time, displacement.subs(vals), 'numpy')
    plt.close('all')
    fig, axs = plt.subplots(3, 1, sharex=True, figsize=(6, 10))
    time_vals = np.linspace(0, 1/vals[2][1], 1000)
    t1_index = np.argmin(np.abs(time_vals - t1_def_val))
    t2_index = np.argmin(np.abs(time_vals - t2_val))

    axs[0].plot(time_vals, aplot(time_vals))
    axs[0].plot(time_vals, ky.subs(vals)*np.ones(len(time_vals)), 'k--', linewidth=0.5)

    axs[0].plot(time_vals, abplot(time_vals))
    axs[0].set_title('Acceleration, g')
    axs[1].plot(time_vals, vplot(time_vals))
    axs[1].plot(time_vals[t1_index:t2_index], vbplot(time_vals)[t1_index:t2_index])
    axs[1].fill_between(time_vals[t1_index:t2_index], vbplot(time_vals)[t1_index:t2_index], vplot(time_vals)[t1_index:t2_index], color='gray', alpha=0.5)
    axs[1].set_title('Velocity, m/s')

    
    axs[2].plot(time_vals, displacementplot(time_vals))
    axs[2].set_title('Displacement, cm')

    if save:
        plt.savefig(f'harmonic_solution_freq-{vals[2][1]}_ky-{vals[1][1]}.png')
    

In [21]:
df = harmonic_solutions(combinations, save=True)

In [22]:
# Group the DataFrame by 'ky_val'
ky_groups = df.groupby('ky (g)')

plt.close('all')
fig, ax = plt.subplots()

# For each group, create a line plot with 'freq_val' as the x-axis and 'harmonic_results' as the y-axis
for name, group in ky_groups:
    ax.semilogy(group['Frequency (Hz)'], group['Displacement (m)'], label=name)

# Set labels and legend
ax.set_xlabel('Input Frequency (Hz)')
ax.set_ylabel('Sliding Block Displacement (m)')
ax.legend(title='Yield Acceleration (g)')
ax.set_title('Analytical solution for rigid block sliding from a single cycle of harmonic input')
ax.grid(True)

# Display the plot
plt.show()